Modulo representación grafica

In [1]:
import FlowRead

Se definen atomos por posición, color y dirección

In [2]:
import Logica
Nx = 5
Ny = 5
Nc = 4
Nd = 7
X = list(range(Nx))
Y = list(range(Ny))
C = list(range(Nc))
D = list(range(Nd))
xypos = []
Colores = {
    0 : 'R',
    1 : 'G',
    2 : 'B',
    3 : 'O' 
}
Direcciones = {
    0 : "t",
    1 : "tb",
    2 : "tl",
    3 : "tr",
    4 : "lb",
    5 : "rb",
    6 : "lr"
}

reglas = []

OenCasilla = Logica.Descriptor([Nx,Ny,Nc,Nd])

def escribir(self,literal):
    if '-' in literal:
        atomo = literal[1:]
        neg = ' no'
    else:
        atomo = literal
        neg = ''
        x, y, c, d = self.inv(atomo)
        return f"{neg}({X[x]},{Y[y]})/{Colores[c]}{Direcciones[d]}"

from types import MethodType
OenCasilla.escribir = MethodType(escribir, OenCasilla)

#### REGLA 1: 

Ninguna Casilla puede estar vacia

In [3]:
def asignarCD():
    Y_xy = []
    for x in X:
        for y in Y:
                Oc = []
                for c in C:
                    if (x,y) not in xypos:
                        cd = [OenCasilla.P([x,y,c,u]) for u in D if u != 0]
                        formula = Logica.Otoria(cd) #si no es terminal se le asigna color y direccion
                        Oc.append(formula)
                if len(Oc) != 0:
                    Y_xy.append(Logica.Otoria(Oc))
    return Logica.Ytoria(Y_xy)



#### REGLA 2: 

Solo un color y dirección por Casilla

In [4]:
def unCD():
    Y_xy = []
    for x in X:
        for y in Y:
            Yd = []
            for d in D:
                Yc = []
                for c in C:
                    ocd = [OenCasilla.P([x,y,u,m]) for u in C for m in D if u != c or m != d] #casillas con diferente direccion o color
                    formula = "("+OenCasilla.P([x,y,c,d])+">-"+Logica.Otoria(ocd)+")"
                    Yc.append(formula)
                Yd.append(Logica.Ytoria(Yc))
            Y_xy.append(Logica.Ytoria(Yd))
    return Logica.Ytoria(Y_xy)

# REGLA 3:

- Si es terminal, debe tener un solo vecino(definido por x+1 x-1 y+1 o y-1)

In [5]:
def vecT():
    Y_xy = []
    for x in X:
        for y in Y:
            Y_c = []
            for c in C:
                vecinos = [OenCasilla.P([u,m,c,d]) for u in X for m in Y for d in D if ((u == x-1 and m == y) or (u == x+1 and m == y) or (m==y+1 and u == x) or (m==y-1 and u == x)) and d!=0 ]
                formula = "("+OenCasilla.P([x,y,c,0])+">"
                v = []
                for e in vecinos:
                    otros_vecinos = [n for n in vecinos if n != e]
                    form = "("+e+"Y-"+Logica.Otoria(otros_vecinos)+")"
                    v.append(form)
                vv = Logica.Otoria(v)
                formula+=vv+")"
                if len(vecinos) != 1:
                    Y_c.append(formula)
                else:
                    Y_c.append("("+OenCasilla.P([x,y,c,0])+">"+vecinos[0]+")")
            Y_xy.append(Logica.Ytoria(Y_c))
    return Logica.Ytoria(Y_xy)


#### INTERPRETACIÓN 

##### LECTURA PARAMETROS INICIALES

In [6]:
imgD = {
	'-' : 'img/empty.gif',
	'B' : 'img/blueend.gif',
	'O' : 'img/orangeend.gif',
	'R' : 'img/redend.gif',
	'G' : 'img/greenend.gif',
	'r' : 'img/lrred.gif',
	'g' : 'img/lrgreen.gif',
	'b' : 'img/lrblue.gif',
	'z' : 'img/tbblue.gif',
	'x' : 'img/tlblue.gif',
	'c' : 'img/trblue.gif',
	'v' : 'img/lbblue.gif',
	'n' : 'img/rbblue.gif',
	'o' : 'img/lrorange.gif',
	'9' : 'img/cirno.gif',
	'Z' : 'img/tborange.gif',
	'X' : 'img/tlorange.gif',
	'C' : 'img/trorange.gif',
	'V' : 'img/lborange.gif',
	'N' : 'img/rborange.gif',
	'q' : 'img/tbgreen.gif',
	'w' : 'img/tlgreen.gif',
	'e' : 'img/trgreen.gif',
	't' : 'img/lbgreen.gif',
	'y' : 'img/rbgreen.gif',
	'Q' : 'img/tbred.gif',
	'W' : 'img/tlred.gif',
	'E' : 'img/trred.gif',
	'T' : 'img/lbred.gif',
	'Y' : 'img/rbred.gif',


}
#lee un archivo con los parametros de la matriz
def FlowRead(MAP_FILE):
	MAP_FILE = open(MAP_FILE,"r")
	MAP_MATRIX = [j.strip() for j in MAP_FILE]
	MAP_FILE.close()
	return MAP_MATRIX
	
#muestra la matriz flow free (una vez aplicado flowread)
def FlowPrint(MAP_MATRIX):
	import turtle
	#recibe una matriz de FlowFree y la muestra
	FlowWindow = turtle.Screen()
	FlowWindow.setup(500,500)
	FlowWindow.title("FlowFree")
	FlowWindow.tracer(0)
	for k in imgD.keys():
   		FlowWindow.addshape(imgD[k])
	cell = turtle.Turtle()
	cell.pu()
	cell.ht()
	y = 200
	for CELL in MAP_MATRIX:
		cell.sety(y)
		x = -200
		for TYPE in CELL:
			cell.setx(x)
			try:
				cell.shape(imgD[TYPE])
			except:
				cell.shape(imgD['9'])
			cell.stamp()
			x += 100
		y-=100 
	FlowWindow.exitonclick()

##### SOLUCION

In [7]:
def decodificar(list):
    matriz = []
    m = []
    pos = {}
    t = {
        "Rt" : "R",
        "Rlr" : "r",
        "Rtb" : "Q",
        "Rtl" : "W",
        "Rtr" :    "E",
        "Rlb" : "mapa",
        "Rrb" : "Y",
        "Gt"  : "G",
        "Glr" : "g",
        "Gtb" : "q",
        "Gtl" : "w",
        "Gtr" : "e",
        "Glb" : "t",
        "Grb" : "y",
        "Bt" : "B",
        "Btb": "z",
        "Btl" : "x",
        "Btr" : "c",
        "Blb" : "v",
        "Brb" : "n",
        "Ot" : "O",
        "Olr" : "o",
        "Otb" : "Z",
        "Otl" : "X",
        "Otr" : "C",
        "Olb" : "V",
        "Orb" : "N"

    }
    for key in list:
        m.append(OenCasilla.escribir(key))
    for i in m:
        pos[(int(i[1]),int(i[3]))] = i.split("/")[1]
    for y in range(Ny):
        strr = ""
        for x in range(Nx):
            for key in pos.keys():
                if key[0] == x and key[1] == y:
                    strr+=t[pos[(x,y)]]
                else:
                    continue
        matriz.append(strr)
    return matriz

def asignarReglas(lista):
    reglas = []
    reglas.append(unCD())
    reglas.append(vecT())
    for i in lista:
        # print(i[0],i[1],i[2],i[3])
        reglas.append(OenCasilla.P([i[0],i[1],i[2],i[3]]))
        xypos.append((i[0],i[1]))
    reglas.append(asignarCD())
    return resolver(Logica.Ytoria(reglas))

def resolver(formula):
    S = Logica.tseitin(formula)
    S,I = Logica.dpll(S,{})
    lis = []
    for k in I:
        if (ord(k) >= OenCasilla.rango[0]) and (ord(k) <= OenCasilla.rango[1]) and I[k]:
            lis.append(k)
    return decodificar(lis)

In [20]:
mapa = FlowRead(input("mapa: "))

In [21]:
terminales = []
for y in range(len(mapa)):
    for x in range(len(mapa[y])):
        if mapa[y][x] == 'R':
            terminales.append((x,y,0,0))
        elif mapa[y][x] == 'G':
            terminales.append((x,y,1,0))
        elif mapa[y][x] == 'B':
            terminales.append((x,y,2,0))
        elif mapa[y][x] == 'O':
            terminales.append((x,y,3,0))
        else:
            continue
formula = asignarReglas(terminales)
FlowPrint(formula)
